In [55]:
import pymongo
import datetime
import pandas as pd
import numpy as np
import cuemath_db as cuedb
import psycopg2
from update_from_cuedb import *


In [4]:
'''
client_mongo = pymongo.MongoClient(cuedb.db_asli_intel['host'])
inteldb = client_mongo[cuedb.db_asli_intel['db_name']]
item_collection = inteldb['Items']
activity_collection = inteldb['Activities']
item_response_collection = inteldb['Item_Responses']
session_collection = inteldb['Sessions']
student_collection = inteldb['Students']
program_collection = inteldb['Programs']
chapter_collection = inteldb['Chapters']
node_collection = inteldb['Nodes']
block_collection = inteldb['Blocks']
classroom_collection = inteldb['Classrooms']
'''
def weird_division(n, d):
    return n / d if d else 0

In [8]:
def get_student_chapter_data():
    conn = cuedb.get_cuemath_redshift_connection()
    cur = conn.cursor()
    std_chp_cols = ['id', 'student_id', 'status', 'assessment_status', 'chapter_version_id', 'started_on', 'completed_on']
    chp_ver_cols = ['chapter_id', 'version']
    chp_cols = ['name', 'code', 'description', 'is_default']
    sql_query = "SELECT "+ ", ".join(['service_concepts_student_chapter.'+s for s in std_chp_cols]) + ", " + \
                            ", ".join(['service_concepts_chapter_version.'+s for s in chp_ver_cols]) + ", " + \
                            ", ".join(['service_concepts_chapter.'+s for s in chp_cols]) + \
                            " FROM service_concepts_student_chapter"+ \
                            " inner join service_concepts_chapter_version on (service_concepts_student_chapter.chapter_version_id = service_concepts_chapter_version.id)" + \
                            " inner join service_concepts_chapter on (service_concepts_chapter_version.chapter_id = service_concepts_chapter.id);"
    print(sql_query)
    cur.execute(sql_query)
    df_module_id_code = pd.DataFrame(cur.fetchall(), columns=std_chp_cols + chp_ver_cols + chp_cols)
    cur.close()
    conn.close()
    return(df_module_id_code)


In [9]:
df = get_student_chapter_data()
df.to_csv('student-chapter.csv')


SELECT service_concepts_student_chapter.id, service_concepts_student_chapter.student_id, service_concepts_student_chapter.status, service_concepts_student_chapter.assessment_status, service_concepts_student_chapter.chapter_version_id, service_concepts_student_chapter.started_on, service_concepts_student_chapter.completed_on, service_concepts_chapter_version.chapter_id, service_concepts_chapter_version.version, service_concepts_chapter.name, service_concepts_chapter.code, service_concepts_chapter.description, service_concepts_chapter.is_default FROM service_concepts_student_chapter inner join service_concepts_chapter_version on (service_concepts_student_chapter.chapter_version_id = service_concepts_chapter_version.id) inner join service_concepts_chapter on (service_concepts_chapter_version.chapter_id = service_concepts_chapter.id);


In [10]:
students = student_collection.collection.find()
with open('./student_list.csv', 'w') as fp:
    fp.write('ID, Name \n')
for std in students:
    with open('./student_list.csv', 'a') as fp:
        fp.write(std.get('_id') +", "+ std.get('first_name', '') + " " +std.get('last_name') + '\n')
    

# Get Data Schema

In [21]:
fl_nm = './Vaibhav/activity_data_dump_04_06_2019.csv'
inteldb = IntelDB('local')
activities = inteldb.activity_collection.collection.find()
with open(fl_nm, "w") as fp:
    fp.write('Program_Code, Chapter_Code, Chapter_Version, Worksheet ID, Reference, #Widgets, Median Time, Accuracy, #Data Points \n')
for idx, activity in enumerate(activities):
    total_widgets = 0
    if(activity.get('distribution_time')):
        num_data_pts = len(activity.get('distribution_time', {}))
        all_times = list(activity.get('distribution_time', {}).values())
        median_time = np.percentile(all_times, 50)/60.0
        all_scores = list(activity.get('distribution_score', {}).values())
        median_score = np.percentile(all_scores, 50)
        nodes_linked = activity.get('nodes_linked', [])
        for item_refs in activity.get('data').get('items'):
            item = inteldb.item_collection.collection.find_one({'_id':item_refs})
            if(item is not None):
                total_widgets += len(item.get('definition').get('widgets'))
        for node_id in nodes_linked:
            node = inteldb.node_collection.collection.find_one({'_id':node_id})
            with open(fl_nm, "a") as fp:
                fp.write(", ".join([
                    str(node.get('program_code', 'NA')),
                    str(node.get('chapter_code', 'NA')),
                    str(node.get('chapter_version', 0)),
                    activity.get('_id'),
                    activity.get('reference'), 
                    str(total_widgets), 
                    str(median_time), 
                    str(median_score/activity.get('max_score')*100), 
                    str(num_data_pts)])+ '\n')


# Student Summaries

In [53]:
#
fl_nm = './Shalini/student_overall_score.csv';
with open(fl_nm, "w") as fp:
    fp.write('ID, Name, Email, Phone, Joined On, Program_Code, #Chapters Attempted, #Chapters Finished, score_classwork, score_homework, score_assessments, score_remedials, time(hrs) \n')
programs = program_collection.collection.find();
student_list = student_collection.collection.find()
attrib_list = ["lessons", "homework", "assessments", "remedials"]
for student in student_list:
    print('student found')
    student_score={}
    student_time={}
    student_chapter_list={}
    student_max_score={}
    for program_id, program in student.get('progress', []).items():
        student_score[program_id]= {}
        student_max_score[program_id] = {}
        student_time[program_id]=0
        student_chapter_list[program_id]={'started':{}, 'completed':{}}
        for attrib in attrib_list:
            student_score[program_id][attrib]=0
            student_max_score[program_id][attrib]=0
            for activityID, activity in student.get(attrib).items():
                for sessionID, session in activity.get('sessions_attempted', {}).items():
                    node = node_collection.collection.find_one({'_id':session.get('node_id')})
                    chapter_code = node.get('chapter_code')
                    if(activity.get('program_id') == program_id):
                        student_chapter_list[program_id]['started'][chapter_code] = 1
                        if(attrib=='assessments'):
                            student_chapter_list[program_id]['completed'][chapter_code] = 1
                        student_time[program_id]+=(session.get('time_spent',0)/3600.0)
                        student_score[program_id][attrib] += (float(session.get('score', 0.0)))
                        student_max_score[program_id][attrib]+=(float(session.get('max_score', 0.0)))
                    
#         print()

        with open(fl_nm, "a") as fp:                                                                                            
            fp.write(", ".join([
                student.get('_id'),
                (student.get('first_name','') + ' ' + student.get('last_name','')),
                student.get('email',''),
                student.get('phone',''),
                str(student.get('created_on')),
                program.get('program_code', ''),
                str(len(student_chapter_list[program_id]['started'])),
                str(len(student_chapter_list[program_id]['completed'])),
                ", ".join([str(weird_division(student_score[program_id][attrib], student_max_score[program_id][attrib])) for attrib in attrib_list]),
                str(student_time[program_id]),
            ])+"\n")

student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
studen

student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
student found
studen

# Get Unfinished Homeworks

In [20]:
session_list = session_collection.collection.find({'activity_type':'HOMEWORK'})
with open('./Vaibhav/homeworks_uncleared.csv', "w") as fp:
    fp.write('Session ID, Node_ID, Date Completed, User_ID, Activity_ID, Session Cleared?, Score, Max_Score, #Widgets, Time(mins) \n')
for session in session_list:
    score = session.get('score')
    max_score = session.get('max_score')
    accuracy = weird_division(score, max_score)
    if(accuracy<0.75):
        with open('./Vaibhav/homeworks_uncleared.csv', "a") as fp:
            fp.write(", ".join([session.get('_id'), session.get('node_id'), str(session.get('dt_completed')), session.get('user_id'), session.get('activity_id'), session.get('session_cleared'), str(session.get('score')), str(session.get('max_score')), str(session.get('num_questions')), str(session.get('time_total')/60.0)]) + '\n')

    

# Get All Students' Progress

In [23]:
# 
inteldb = IntelDB('dev')
fl_nm = './Shalini/student_overall_progress_june.csv';
with open(fl_nm, "w") as fp:
    fp.write('Student ID, Name, Email, Phone, Program_Code, Classes Attended, Total Classes, Chapter Code, WS type, WS Reference, #Items, Session_Cleared?, Dt_Completed, score, max-score, accuracy, time(mins) \n')
student_list = inteldb.student_collection.collection.find()
attrib_list = ["lessons", "homework", "assessments", "remedials"]
for student in student_list:
    num_classes = inteldb.classroom_collection.collection.count_documents({
            'student_id':student.get('_id'), 
            'status':'P',
            'sessions_attempted':{'$ne':{}},
            'date':{'$lte':'2019-07-01', '$gt':'2019-05-30'}})
    num_total_classes = inteldb.classroom_collection.collection.count_documents({'student_id':student.get('_id'), 'date':{'$lte':'2019-07-01', '$gt':'2019-05-31'}})
    if(num_classes<8):
        continue
    student_score={}
    student_time={}
    student_chapter_list={}
    student_max_score={}
    for program_id, program in student.get('progress', []).items():
        for attrib in attrib_list:
            for activityID, activity_summary in student.get(attrib).items():
                if(activity_summary.get('program_id') == program_id):
                    activity = inteldb.activity_collection.collection.find_one({'_id':activityID})
                    for sessionID, session_summary in activity_summary.get('sessions_attempted', {}).items():
                        node = inteldb.node_collection.collection.find_one({'_id':session_summary.get('node_id')})
                        dt_cmp_raw = session_summary.get('dt_completed', 'NA')
                        if(dt_cmp_raw!='NA' and dt_cmp_raw!='NaT'):
                            #print(type(dt_cmp_raw))
                            dt_completed = datetime.datetime.strptime(dt_cmp_raw, "%Y-%m-%d %H:%M:%S.%f")
                        else:
                            dt_completed = 'NA'
                        if(dt_completed != 'NA' and dt_completed > datetime.datetime(2019, 5, 30, 0, 0, 0, 0) and dt_completed < datetime.datetime(2019, 7, 1, 0, 0, 0, 0)):
                            with open(fl_nm, "a") as fp:                                                                                            
                                fp.write(", ".join([
                                    student.get('_id'),
                                    (student.get('first_name','') + ' ' + student.get('last_name','')),
                                    student.get('email',''),
                                    student.get('phone',''),
                                    str(node.get('program_code', 'NA')),
                                    str(num_classes),
                                    str(num_total_classes),
                                    str(node.get('chapter_code', 'NA')),
                                    node.get('type', 'NA'),
                                    activity.get('reference', 'NA'),
                                    str(len(activity.get('data', []).get('items', []))),
                                    str(session_summary.get('session_cleared', False)),
                                    str(dt_completed),
                                    str(session_summary.get('score', 0.0)),
                                    str(session_summary.get('max_score', 0.0)),
                                    str(session_summary.get('score', 0.0)/session_summary.get('max_score', 1.0)),
                                    str(session_summary.get('time_spent',0)/60.0)
                                ])+"\n")

# Get Homework Attempts

In [26]:
inteldb = IntelDB('dev')
student_list = inteldb.student_collection.collection.find()
fl_nm = './Vaibhav/homework_attempts.csv'
with open(fl_nm, "w") as fp:
    fp.write('Activity ID, Activity Reference, #Cleared in 1 attempt, #Cleared in 2 attempt, #Cleared in 3 attempt, #Cleared in 4 attempt, #Cleared in 5 attempt, Not Cleared,  \n')

activity_dict = {}
for student in student_list:
    for activity_id, activity_summary_obj in student.get('homework').items():
        if(not activity_dict.get(activity_id)):
            activity_dict[activity_id] = {'reference':activity_summary_obj.get('activity_reference'), '1':0, '2':0, '3':0, '4':0, '5':0, 'unf':0}
        if(activity_summary_obj.get('unlock_status')):
            num_attempts = len(activity_summary_obj.get('sessions_attempted', {}))
            if(activity_summary_obj.get('complete_status')):
                if(activity_dict[activity_id].get(str(num_attempts))):
                    activity_dict[activity_id][str(num_attempts)] += 1
                else:
                    activity_dict[activity_id][str(num_attempts)] = 1
            elif(num_attempts>0):
                activity_dict[activity_id]['unf'] += 1

for activity_id, activity_details in activity_dict.items():
    with open(fl_nm, "a") as fp:
        fp.write(", ".join([activity_id, activity_details.get('reference', ''), str(activity_details.get('1', 0)), str(activity_details.get('2', 0)), str(activity_details.get('3', 0)), str(activity_details.get('4', 0)), str(activity_details.get('5', 0)), str(activity_details.get('unf', 0))]) + '\n')

    

# Get Chapter Data

In [11]:
chapter_code = 'CBSE.G10.REANM2'
inteldb = IntelDB('prod')
fl_nm = './Manish/chapter_realnm2_data.csv'
with open(fl_nm, "w") as fp:
    fp.write('Program_Code, Chapter_Code, Chapter_Version, Position, Worksheet ID, Reference, #Widgets, Median Time, Accuracy, #Data Points \n')
node_cursor = inteldb.node_collection.collection.find({'chapter_code':chapter_code})
activity_id_list, node_dict = [], {}
for node in node_cursor:
    node_dict[node.get('_id')] = node
    activity_id_list.append(node.get('activity_id'))
activities = inteldb.activity_collection.collection.find({'_id':{'$in':activity_id_list}})
for idx, activity in enumerate(activities):
    total_widgets = 0
    if(activity.get('distribution_time')):
        print(activity.get('distribution_time', {}).keys())
        num_data_pts = len(activity.get('distribution_time', {}))
        all_times = list(activity.get('distribution_time', {}).values())
        median_time = np.percentile(all_times, 50)/60.0
        all_scores = list(activity.get('distribution_score', {}).values())
        median_score = np.percentile(all_scores, 50)
        nodes_linked = activity.get('nodes_linked', [])
        for item_refs in activity.get('data').get('items'):
            item = inteldb.item_collection.collection.find_one({'_id':item_refs})
            if(item is not None):
                total_widgets += len(item.get('definition').get('widgets'))
        for node_id in nodes_linked:
            node = node_dict[node_id]
            with open(fl_nm, "a") as fp:
                fp.write(", ".join([
                    str(node.get('program_code', 'NA')),
                    str(node.get('chapter_code', 'NA')),
                    str(node.get('chapter_version', 0)),
                    str(node.get('position_in_chapter', 0)),
                    activity.get('_id'),
                    activity.get('reference'), 
                    str(total_widgets), 
                    str(median_time), 
                    str(median_score/activity.get('max_score')*100), 
                    str(num_data_pts)])+ '\n')
    
    

dict_keys(['302b13d2-6049-11e9-a417-02420a00f6a0', 'ccd5582e-d12a-11e8-be42-02420a00002d', 'f9e9a944-ebda-11e8-8688-02420a0000ae', 'd1013b22-cbb0-11e8-ac8a-02420a00002d', '680bde5e-5dac-11e9-be1b-02420a00f696', '101be654-d764-11e8-9615-02420a0000ad'])
dict_keys(['ccd5582e-d12a-11e8-be42-02420a00002d', 'f9e9a944-ebda-11e8-8688-02420a0000ae', 'd1013b22-cbb0-11e8-ac8a-02420a00002d', '680bde5e-5dac-11e9-be1b-02420a00f696', '101be654-d764-11e8-9615-02420a0000ad'])
dict_keys(['ccd5582e-d12a-11e8-be42-02420a00002d', '680bde5e-5dac-11e9-be1b-02420a00f696', 'd1013b22-cbb0-11e8-ac8a-02420a00002d', 'f9e9a944-ebda-11e8-8688-02420a0000ae', '101be654-d764-11e8-9615-02420a0000ad'])
dict_keys(['ccd5582e-d12a-11e8-be42-02420a00002d', 'f9e9a944-ebda-11e8-8688-02420a0000ae', 'd1013b22-cbb0-11e8-ac8a-02420a00002d', '680bde5e-5dac-11e9-be1b-02420a00f696', '101be654-d764-11e8-9615-02420a0000ad'])
dict_keys(['ccd5582e-d12a-11e8-be42-02420a00002d', 'd1013b22-cbb0-11e8-ac8a-02420a00002d', '101be654-d764-11e8-9

# Get Student List

In [20]:
inteldb = IntelDB('prod')
students = inteldb.student_collection.collection.find({'current_program':'TRNGUS.G6'})
fl_nm = './Sibi/TRNGUS_Student_List.csv'
with open(fl_nm, "w") as fp:
    fp.write('Student ID, Program, Name, phone, email, link \n')
for student in students:
    with open(fl_nm, "a") as fp:
        fp.write(", ".join([student.get('_id'), 
                            student.get('current_program'), 
                            student.get('first_name', '') + ' ' + student.get('last_name', ''),
                            str(student.get('phone', '')),
                            student.get('email', ''),
                            "https://leap-teacher.cuemath.com/student-profile/"+ student.get('_id') +"/analytics"
                           ]) + '\n')

# Session Dump

In [34]:
inteldb = IntelDB('prod')
sessions = inteldb.session_collection.collection.find({'completion':1})
fl_nm = './Rashmi/session_dump.csv'
with open(fl_nm, "w") as fp:
    fp.write('Session ID, User ID, Chapter Code, Worksheet ID, Worksheet Type, Score, Max Score, Number of Questions Attempted, Total Number of Questions, Total Time on Worksheets \n')
for session in sessions:
    if(session.get('completion')==1):
        if(session.get('activity_chapter_code')):
            chapter_code = session.get('activity_chapter_code')
        else:
            chapter_code = 'NA'
        with open(fl_nm, "a") as fp:
            fp.write(", ".join([session.get('_id'),
                                session.get('user_id'),
                                chapter_code,
                                session.get('activity_id'),
                                session.get('activity_type'),
                                str(session.get('score')),
                                str(session.get('max_score')),
                                str(session.get('num_attempted')),
                                str(session.get('num_questions')),
                                str(session.get('time_total')),
                           ]) + '\n')


# Get Itemwise Analysis

In [45]:
inteldb = IntelDB('prod')
activity_id='d035e64a-5c2a-11e9-973a-c4b301c785cf'
item_responses = inteldb.item_response_collection.collection.find({'activity_id':activity_id})
data_dict = {}
for item_response in item_responses:
    print(item_response.get('_id'))
    if(data_dict.get(item_response.get('item_id'))):
        data_dict[item_response.get('item_id')].append(item_response.get('time'))
    else:
        data_dict[item_response.get('item_id')] = [item_response.get('time')]
        

f177c79e-611b-11e9-baaf-02420a000347_Proportion As Equivalence Of Ratios 205
f177c79e-611b-11e9-baaf-02420a000347_Proportion As Equivalence Of Ratios 204
f177c79e-611b-11e9-baaf-02420a000347_Proportion As Equivalence Of Ratios 208
f177c79e-611b-11e9-baaf-02420a000347_Proportion As Equivalence Of Ratios 202
f177c79e-611b-11e9-baaf-02420a000347_Proportion As Equivalence Of Ratios 203
f177c79e-611b-11e9-baaf-02420a000347_Proportion As Equivalence Of Ratios 206
f177c79e-611b-11e9-baaf-02420a000347_Proportion As Equivalence Of Ratios 207
f177c79e-611b-11e9-baaf-02420a000347_Proportions as Equivalence of Ratios 211
f177c79e-611b-11e9-baaf-02420a000347_The Unitary Method 201
f177c79e-611b-11e9-baaf-02420a000347_The Unitary Method 202
f177c79e-611b-11e9-baaf-02420a000347_The Unitary Method 203
f177c79e-611b-11e9-baaf-02420a000347_The Unitary Method 204


In [46]:
print(data_dict)

{'Proportion As Equivalence Of Ratios 205': [122], 'Proportion As Equivalence Of Ratios 204': [12], 'Proportion As Equivalence Of Ratios 208': [827], 'Proportion As Equivalence Of Ratios 202': [133], 'Proportion As Equivalence Of Ratios 203': [93], 'Proportion As Equivalence Of Ratios 206': [14], 'Proportion As Equivalence Of Ratios 207': [32], 'Proportions as Equivalence of Ratios 211': [75], 'The Unitary Method 201': [30], 'The Unitary Method 202': [99], 'The Unitary Method 203': [114], 'The Unitary Method 204': [200]}


# Classroom Usage Data

In [100]:
inteldb = IntelDB('prod')
classrooms = inteldb.classroom_collection.collection.find({'av_data':{'$ne':{}}}).sort('date')
fl_nm = './Govind/classroom_data.csv'
with open(fl_nm, 'w') as fp:
    fp.write(",".join(
            ['ID', 'Date',
             'Slot', 
             'Teacher_ID', 
             'Student_ID', 
             'Total Call Duration(mins)',
             'Number of Calls',
             'Average Call Lag (s)',
             'Number of Doubts Raised',
             'Number of Sheets Attempted',
             'Number of Questions Attempted',
             'Total time spent on worksheets(mins)',
             'Classroom Rating',
             'Feedback Data',
            ]) + '\n')
for cls in classrooms:
    av_data = cls.get('av_data')
    total_call_dur = 0.0
    num_calls = 0
    total_stream_lag = 0.0
    nm_lg_available = 0
    total_time_on_ws = 0.0
    num_sheets_attempted = 0
    num_items_attempted = 0
    for call in av_data.get('call_data'):
        total_call_dur += float(call.get('duration_in_seconds'))/60.0
        num_calls += 1
        if(not pd.isnull(call.get('call_stream_lag')) and call.get('call_stream_lag')>0):
            total_stream_lag += call.get('call_stream_lag')
            nm_lg_available += 1

    num_doubts = sum([dbt.get('num_button_press') for dbt in av_data.get('doubts_raised').values()])
    if(nm_lg_available == 0):
        avg_call_lag = 'NA'
    else:
        avg_call_lag = total_stream_lag/nm_lg_available
    for session in cls.get('sessions_attempted').values():
        num_items_attempted += session.get('num_attempted_today')
        num_sheets_attempted += 1
        total_time_on_ws += session.get('time_spent')/60.0
    rt = cls.get('ratings', {})
    rating = rt.get('rating', 'NA')
    feedback = 'NA'
    slot = cls.get('scheduled_start_time', '') + '-' + cls.get('scheduled_end_time', '')
    if(rt.get('meta') and rt.get('meta').startswith('{')):
        feedback = ", ".join(json.loads(rt.get('meta', '[]')).get('feedback_tags', []))
    row = ",".join([
        cls.get('_id'),
        cls.get('date'),
        slot,
        cls.get('teacher_id'),
        cls.get('student_id'),
        str(total_call_dur),
        str(num_calls),
        str(avg_call_lag),
        str(num_doubts),
        str(num_sheets_attempted),
        str(num_items_attempted),
        str(total_time_on_ws),
        str(rating),
        feedback,
    ])+'\n'
    with open(fl_nm, 'a') as fp:
        if(slot != '00:15:00-23:45:00'):
            print(cls.get('date'), slot, '00:15:00-23:45:00', slot == '00:15:00-23:45:00')
            fp.write(row)

2018-10-01 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-10-01 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-10-01 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-10-01 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-10-03 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-10-03 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-10-03 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-10-03 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-10-05 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-10-05 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-10-05 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-10-06 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-10-08 15:00:00-16:00:00 00:15:00-23:45:00 False
2018-10-08 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-10-08 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-10-08 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-10-08 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-10-08 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-10-09 20:00:00-21:00:00 00:15:00-23:45:00

2018-11-19 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-11-19 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-11-19 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-11-19 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-11-19 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-11-19 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-11-19 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-11-20 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-11-20 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-11-20 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-11-20 16:15:00-17:15:00 00:15:00-23:45:00 False
2018-11-20 15:00:00-16:00:00 00:15:00-23:45:00 False
2018-11-20 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-11-20 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-11-20 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-11-20 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-11-20 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-11-20 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-11-20 20:00:00-21:00:00 00:15:00-23:45:00

2018-12-12 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-12-12 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-12-12 20:00:00-21:00:00 00:15:00-23:45:00 False
2018-12-12 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-12-12 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-12-12 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-12-12 15:00:00-16:00:00 00:15:00-23:45:00 False
2018-12-12 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-12-12 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-12-13 16:15:00-17:15:00 00:15:00-23:45:00 False
2018-12-13 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-12-13 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-12-13 16:15:00-17:15:00 00:15:00-23:45:00 False
2018-12-13 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-12-13 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-12-13 20:00:00-21:00:00 00:15:00-23:45:00 False
2018-12-13 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-12-13 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-12-13 18:45:00-19:45:00 00:15:00-23:45:00

2018-12-31 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-12-31 15:00:00-16:00:00 00:15:00-23:45:00 False
2018-12-31 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-12-31 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-12-31 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-12-31 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-12-31 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-12-31 20:00:00-21:00:00 00:15:00-23:45:00 False
2018-12-31 18:45:00-19:45:00 00:15:00-23:45:00 False
2018-12-31 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-12-31 17:30:00-18:30:00 00:15:00-23:45:00 False
2018-12-31 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-01-02 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-01-02 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-01-02 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-01-02 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-01-02 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-01-02 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-01-02 18:45:00-19:45:00 00:15:00-23:45:00

2019-01-17 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-01-17 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-01-17 16:15:00-17:15:00 00:15:00-23:45:00 False
2019-01-17 10:30:00-11:30:00 00:15:00-23:45:00 False
2019-01-17 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-01-17 13:30:00-14:30:00 00:15:00-23:45:00 False
2019-01-17 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-01-18 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-01-18 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-01-18 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-01-18 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-01-18 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-01-18 15:00:00-16:00:00 00:15:00-23:45:00 False
2019-01-18 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-01-18 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-01-18 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-01-18 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-01-18 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-01-18 17:30:00-18:30:00 00:15:00-23:45:00

...

2019-06-28 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-06-28 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-06-28 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-06-28 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-06-28 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-06-28 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-06-28 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-06-28 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-06-28 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-06-28 05:00:00-05:50:00 00:15:00-23:45:00 False
2019-06-28 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-06-28 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-06-28 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-06-28 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-06-28 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-06-28 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-06-28 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-06-28 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-06-28 20:00:00-21:00:00 00:15:00-23:45:00

2019-07-01 16:15:00-17:15:00 00:15:00-23:45:00 False
2019-07-01 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-01 15:00:00-16:00:00 00:15:00-23:45:00 False
2019-07-01 16:15:00-17:15:00 00:15:00-23:45:00 False
2019-07-01 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-07-01 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-01 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-07-01 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-07-01 15:00:00-16:00:00 00:15:00-23:45:00 False
2019-07-01 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-01 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-07-01 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-01 16:15:00-17:15:00 00:15:00-23:45:00 False
2019-07-01 16:15:00-17:15:00 00:15:00-23:45:00 False
2019-07-01 15:00:00-16:00:00 00:15:00-23:45:00 False
2019-07-01 16:15:00-17:15:00 00:15:00-23:45:00 False
2019-07-01 05:00:00-05:50:00 00:15:00-23:45:00 False
2019-07-01 16:15:00-17:15:00 00:15:00-23:45:00 False
2019-07-01 18:45:00-19:45:00 00:15:00-23:45:00

2019-07-03 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-07-03 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-03 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-07-03 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-03 06:10:00-07:00:00 00:15:00-23:45:00 False
2019-07-03 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-07-03 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-03 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-03 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-07-03 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-03 15:00:00-16:00:00 00:15:00-23:45:00 False
2019-07-03 16:15:00-17:15:00 00:15:00-23:45:00 False
2019-07-03 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-03 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-03 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-03 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-07-03 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-07-03 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-07-03 17:30:00-18:30:00 00:15:00-23:45:00

2019-07-04 16:15:00-17:15:00 00:15:00-23:45:00 False
2019-07-04 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-07-04 15:00:00-16:00:00 00:15:00-23:45:00 False
2019-07-04 15:00:00-16:00:00 00:15:00-23:45:00 False
2019-07-04 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-04 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-07-04 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-04 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-04 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-07-04 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-07-04 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-05 17:30:00-18:30:00 00:15:00-23:45:00 False
2019-07-05 16:15:00-17:15:00 00:15:00-23:45:00 False
2019-07-05 16:15:00-17:15:00 00:15:00-23:45:00 False
2019-07-05 18:45:00-19:45:00 00:15:00-23:45:00 False
2019-07-05 15:00:00-16:00:00 00:15:00-23:45:00 False
2019-07-05 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-07-05 20:00:00-21:00:00 00:15:00-23:45:00 False
2019-07-05 18:45:00-19:45:00 00:15:00-23:45:00

TypeError: 'NoneType' object is not iterable

# Get useless students

In [ ]:
inteldb = IntelDB('prod')
students = inteldb.student_collection.collection.find({'assessments':{}, 'is_demo':False})
for student in students:
    classes = inteldb.classroom_collection.collection.find({'status':'P', 'student_id':student.get('_id')})
    teacher_list = set()
    num_classes = 0
    for cls in classes:
        teacher = inteldb.teacher_collection.collection.find_one({'_id':cls.get('teacher_id')})
        try:
            teacher_list.add(teacher.get('first_name', '') + ' ' + teacher.get('last_name', ''))
        except AttributeError:
            pass
        num_classes += 1
    if(num_classes>8):
        print(student.get('first_name', '') + ' ' + student.get('last_name', ''), 'Teacher: ', teacher_list) 

# Session Dump to Google Sheet

In [66]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from curriculum_google_sheet import initialize_creds

from update_from_cuedb import *
import numpy as np

fl_nm_pickle = './google_sheets/token.pickle'
inteldb = IntelDB('prod')

In [64]:
def get_session_dump():
    output_data = [['Session ID', 
                    'Date Started',
                    'Date Finished',
                    'User ID', 
                    'Chapter Code', 
                    'Worksheet ID', 
                    'Worksheet Type', 
                    'Score', 
                    'Max Score', 
                    'Number of Questions Attempted', 
                    'Total Number of Questions', 
                    'Total Time on Worksheets']]
    sessions = inteldb.session_collection.collection.find({'dt_completed':{
                                                            '$gte':'2019-05-31', 
                                                            }, 
                                                           'dt_started':{
                                                            '$lte':'2019-07-01'
                                                            }})
    for session in sessions:
        if(session.get('completion')==1):
            if(session.get('activity_chapter_code')):
                chapter_code = session.get('activity_chapter_code')
            else:
                chapter_code = 'NA'
            output_data.append([session.get('_id'),
                                    session.get('dt_started'),
                                    session.get('dt_completed'),
                                    session.get('user_id'),
                                    chapter_code,
                                    session.get('activity_id'),
                                    session.get('activity_type'),
                                    str(session.get('score')),
                                    str(session.get('max_score')),
                                    str(session.get('num_attempted')),
                                    str(session.get('num_questions')),
                                    str(session.get('time_total')),
                               ])
    return output_data


In [72]:
    def get_session_dump_by_activity_list(activity_ref_list):
    output_data = [['Session ID', 
                    'Date Started',
                    'Date Finished',
                    'Student ID',
                    'Name',
                    'Phone',
                    'Email',
                    'Worksheet Reference', 
                    'Worksheet Type',
                    'Completion',
                    'Score', 
                    'Max Score', 
                    'Number of Questions Attempted', 
                    'Total Number of Questions', 
                    'Total Time on Worksheets']]
    sessions = inteldb.session_collection.collection.find({'activity_reference':{'$in':activity_ref_list}})
    student_id_list = []
    all_students, all_sessions = {}, {}
    for ssn in sessions:
        all_sessions[ssn.get('_id')] =ssn
        student_id_list.append(ssn.get('user_id'))
    students = inteldb.student_collection.collection.find({'_id':{'$in':student_id_list}})
    for std in students:
        all_students[std.get('_id')] =std
    for session_id, session in all_sessions.items():
        student = all_students.get(session.get('user_id'), {})
        output_data.append([session.get('_id'),
                                    str(session.get('dt_started')),
                                    str(session.get('dt_completed')),
                                    session.get('user_id'),
                                    student.get('first_name', 'NA') + ' ' + student.get('last_name', 'NA'),
                                    student.get('phone', 'NA'),
                                    student.get('email', 'NA'),
                                    session.get('activity_reference'),
                                    session.get('activity_type'),
                                    session.get('completion'),
                                    str(session.get('score')),
                                    str(session.get('max_score')),
                                    str(session.get('num_attempted')),
                                    str(session.get('num_questions')),
                                    str(session.get('time_total')/60.0),
                               ])
    return output_data


In [76]:
spreadsheet_id_school_pre_test = '14tW1fiXJNcXx2VRbX91kEUNdv7T0FAi2-wmKnO5WTsc'
spreadsheet_id_us_test ='1pFLAWo0ekBLkj5AVXw76F-wBhi2zW6CVYfCY0gQA7No'
creds = initialize_creds()
service = build('sheets', 'v4', credentials=creds)
activity_ref_list_school_pre_test = ['pre-test-identities', 'School-Pre-Test-Fractions-Decimals']
activity_ref_list_us_test = ['Columbus-Teacher-Test-Section-1', 'Columbus-Teacher-Test-Section-2']
data = get_session_dump_by_activity_list(activity_ref_list_us_test)
final_request = {
            "value_input_option": "USER_ENTERED",
            "data": [{
                    "range": "Raw Data!A1:P"+str(len(data)+3),
                    "values": data
                }]
            }
request = service.spreadsheets().values().batchUpdate(spreadsheetId=spreadsheet_id_us_test, body=final_request)
response = request.execute()
print('Response', response)

Response {'spreadsheetId': '1pFLAWo0ekBLkj5AVXw76F-wBhi2zW6CVYfCY0gQA7No', 'totalUpdatedRows': 758, 'totalUpdatedColumns': 15, 'totalUpdatedCells': 11370, 'totalUpdatedSheets': 1, 'responses': [{'spreadsheetId': '1pFLAWo0ekBLkj5AVXw76F-wBhi2zW6CVYfCY0gQA7No', 'updatedRange': "'Raw Data'!A1:O758", 'updatedRows': 758, 'updatedColumns': 15, 'updatedCells': 11370}]}
